# Porting LSST test queries to Stripe82++ : part 3

Notebook with queries representing the LSST test queries, but against the <a href="http://cas.sdss.org/stripe82/en/" target="_blank">Stripe82 database</a>. The latter was extended with a 'match' and 'coadd' table following script in 
https://docs.google.com/document/d/1qFOE1s4bQUDM8W7wLzoxb2OK1LaPwzd5Lmy_GKUYut0.

Queries from https://dev.lsstcorp.org/trac/wiki/db/queries, third batch:
* Select all variable objects of a specific type 
* Random sample of the data 
* Find all objects similar to the colors of a quasar with redshift in a given range 
* Find quasars 
* Find stars with multiple measurements and with certain magnitude variations 
* Select all objects with certain variability or period or amplitude 
* Low-z QSO candidates using the color cuts 
* Select all "new" objects 
* The BRG sample. Create a volume-limited sample of luminous red galaxies with ? < z < ?  NOT IN QSERV 
* Return moving objects whose motion uncertainty is smaller than measured motion 
* Find all galaxies brighter than given magnitude 
* Find all galaxies with a deVaucouleours profile (rÂ¼ falloff of intensity on disk) and the photometric colors consistent with an elliptical galaxy  NOT IN QSERV 
* Find extremely red galaxies 
* Find high proper motion white dwarf candidates 
* Select objects with proper motion 
* Select objects with a measured parallax 
* ...

Assuming schema S12_sdss from https://lsst-web.ncsa.illinois.edu/schema/index.php?sVer=S12_sdss

Ths notebook is in https://github.com/idies/jhu-lsst/blob/master/Stripe82/test_queries/LSST-Test-Queries%20-%20Part%203.ipynb

In [1]:
import pandas
import matplotlib.pyplot as plt
import numpy as np
import skimage.io                   # image processing library

In [2]:
import SciServer.CasJobs as cj

In [3]:
import SciServer.SkyServer as skyS

In [4]:
# define some variable to be used in query execution

TARGET_CONTEXT="Stripe82_016"

In [5]:
# SLOW
# We should add more columns to coadd
sql="""
SELECT TOP 16 p.objId,c.ra,c.dec,p.petror90_r
  FROM test.coadd as c
  inner join photoobjall AS p
  on c.cobjid=p.objid
  and c.type=3
  and c.u BETWEEN 0 AND 19.6
  AND c.g BETWEEN 0 AND 17
  AND p.petror90_r > 10
"""
df=cj.executeQuery(sql,TARGET_CONTEXT)
df.head(3)

,objId,ra,dec,petror90_r
0,8647474690327707860,-15.410634,-1.183950,14.83797
1,8647474690330001465,-10.275765,-1.060240,23.09696
2,8647474690340618618,14.095148,-1.209754,13.53837


In [6]:
OBJECT_ID=df['objId'][0]

### <a href="https://dev.lsstcorp.org/trac/wiki/db/queries/002" target="_blank">Select all variable objects of a specific type </a>

https://dev.lsstcorp.org/trac/wiki/db/queries/002

QSERV
```sql
SELECT *
FROM   Object
JOIN   _ObjectToType USING(objectId)
JOIN   ObjectType USING (typeId)
WHERE  description = 'Supernova'
  AND  variability > 0.8
  AND  probability > 0.8
```

Uses <a href="https://lsst-web.ncsa.illinois.edu/schema/index.php?sVer=S12_sdss&t=ObjectType" target="_blank">ObjectType</a> table. Assuming Stripe82 has something similar, though with far fewer entries most likely, only the 

In [ ]:

sql="""
select top 10 o.*
  from test.coadd c 
 where c.type=dbo.fPhotoType('Star') 
""" 
df=cj.executeQuery(sql,TARGET_CONTEXT)
df.head()


# use fPhotoType as alternative to _ObjectToType
# Stripe82 does not yet have an ObjectType table, but can be added